## import some module

In [1]:
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense

from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import Evaluation
from pipeline.component import HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.interface import Model
from pipeline.utils.tools import load_job_config

## upload data

In [2]:
# pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)
# partition=4


# dense_data_guest = {"name": "breast_hetero_guest", "namespace": f"experiment"}
# dense_data_host = {"name": "breast_hetero_host", "namespace": f"experiment"}
# tag_data = {"name": "breast_hetero_host", "namespace": f"experiment"}

# import os
# data_base="/data/projects/fate"
# pipeline_upload.add_upload_data(file=os.path.join(data_base, "examples/data/breast_hetero_guest.csv"),
#                                 table_name=dense_data_guest["name"],             # table name
#                                 namespace=dense_data_guest["namespace"],         # namespace
#                                 head=1, partition=partition)               # data info

# pipeline_upload.add_upload_data(file=os.path.join(data_base, "examples/data/breast_hetero_host.csv"),
#                                 table_name=dense_data_host["name"],
#                                 namespace=dense_data_host["namespace"],
#                                 head=1, partition=partition)

# pipeline_upload.add_upload_data(file=os.path.join(data_base, "examples/data/breast_hetero_host.csv"),
#                                 table_name=tag_data["name"],
#                                 namespace=tag_data["namespace"],
#                                 head=1, partition=partition)

# pipeline_upload.upload(drop=1)

## Define some component

### Define reader ,datatransform and intersection

In [3]:
namespace=""
guest = 9999
host = 9999
guest_train_data = {"name": "breast_hetero_guest", "namespace": f"experiment{namespace}"}
host_train_data = {"name": "breast_hetero_host", "namespace": f"experiment{namespace}"}

pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='guest', party_id=guest).component_param(table=guest_train_data)
reader_0.get_party_instance(role='host', party_id=host).component_param(table=host_train_data)

data_transform_0 = DataTransform(name="data_transform_0")
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(with_label=True)
data_transform_0.get_party_instance(role='host', party_id=host).component_param(with_label=False)

intersection_0 = Intersection(name="intersection_0")

### Define hetero nn

In [4]:
hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=100,
                       interactive_layer_lr=0.15, batch_size=-1, early_stop="diff")


guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
                                            
guest_nn_0.add_bottom_model(Dense(units=3, input_shape=(10,), activation="relu",
                                  kernel_initializer=initializers.Constant(value=1)))
                                            
guest_nn_0.set_interactve_layer(Dense(units=2, input_shape=(2,),
                                      kernel_initializer=initializers.Constant(value=1)))
                                            
guest_nn_0.add_top_model(Dense(units=1, input_shape=(2,), activation="sigmoid",
                               kernel_initializer=initializers.Constant(value=1)))


host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)
                                            
host_nn_0.add_bottom_model(Dense(units=3, input_shape=(20,), activation="relu",
                                 kernel_initializer=initializers.Constant(value=1)))
                                            
host_nn_0.set_interactve_layer(Dense(units=2, input_shape=(2,),
                                     kernel_initializer=initializers.Constant(value=1)))
      
                                            
                                            
hetero_nn_0.compile(optimizer=optimizers.SGD(lr=0.15), loss="binary_crossentropy")



hetero_nn_1 = HeteroNN(name="hetero_nn_1")

evaluation_0 = Evaluation(name="evaluation_0")

## add component and start training

In [5]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersection_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_nn_0, data=Data(train_data=intersection_0.output.data))
pipeline.add_component(hetero_nn_1, data=Data(test_data=intersection_0.output.data),
                       model=Model(model=hetero_nn_0.output.model))
pipeline.add_component(evaluation_0, data=Data(data=hetero_nn_0.output.data))

pipeline.compile()

pipeline.fit()

print(hetero_nn_0.get_config(roles={"guest": [guest],
                                    "host": [host]}))
print(pipeline.get_component("hetero_nn_0").get_summary())

2022-08-25 14:13:48.358 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202208251413424383660

2022-08-25 14:13:48.518 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-08-25 14:13:49.066 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
m2022-08-25 14:13:56.660 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2022-08-25 14:13:56.663 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:08
2022-08-25 14:13:57.232 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:08
2022-08-25 14:13:57.806 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:09
2022-08-25 14:13:58.501 | INFO     | pi

{'common': {'hetero_nn_0': {'epochs': 100, 'interactive_layer_lr': 0.15, 'batch_size': -1, 'early_stop': 'diff', 'optimizer': {'learning_rate': 0.15, 'decay': 0.0, 'momentum': 0.0, 'nesterov': False, 'optimizer': 'SGD'}, 'bottom_nn_define': None, 'top_nn_define': None, 'interactive_layer_define': None, 'config_type': 'keras', 'loss': 'binary_crossentropy'}}, 'role': {'guest': {'0': {'hetero_nn_0': {'bottom_nn_define': {'class_name': 'Sequential', 'config': {'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 10], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'dense_input'}}, {'class_name': 'Dense', 'config': {'name': 'dense', 'trainable': True, 'batch_input_shape': [None, 10], 'dtype': 'float32', 'units': 3, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'Constant', 'config': {'value': 1}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularize